# 1주차 과제 - 김민회


## Q1) Type 1, Type 2 Error 비교 (FP, FN의 관점에서)
[자료 링크](https://sumniya.tistory.com/26)

In [1]:
from IPython.display import Image

![예시](img/threshold.png)

위 경우 threshold가 0.3으로 낮고, 그로 인해 acceptance rated가 높다.  
따라서 FP가 높고 FN이 낮다.  
FP는 2종 오류, FN은 1종 오류와 같은 개념이므로 1종 오류는 낮지만 2종 오류는 높다.  
### A) Type 1 Error < Type 2 Error

---

## Q2-1) Accuracy, Precision and Recall (confusion matrix 관점)

![혼동행렬](img/confusion_matrix.png)

Accracy : 모든 분류 결과 중에서 옳게 분류한 비율, 즉 정답/전체  
Precision : True로 분류한 것들 중 옳게 분류한 비율  
Recall : 실제 값이 True인 것 중 모델이 True로 분류한 비율  
### $Accracy   = \frac{TP + TN}{TP + TN + FP+ FN}$  
### $Precision   = \frac{TP}{TP + FP}$  
### $Recall   = \frac{TP}{TP + FN}$  

---

## Q2-2) Precision and Recall 예시 

#### 신약 개발 예시  
제약 회사에서 신약을 개발했을 때 기존 약보다 성능 향상이 유의미한가  
True를 '신약 출시가 합리적'이라 했을 때, FP가 더 치명적이므로 Precision에 더 집중할 것  
이 경우 threshold를 높이는 것은 FP가 낮아지게 하고, Precision은 높아질 것이다. 

---

## Q3) MNIST 코드 따라해보기

[코드 출처](https://www.geeksforgeeks.org/identifying-handwritten-digits-using-logistic-regression-pytorch/)

In [2]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable

In [3]:
# MNIST Dataset (Images and Labels) 
train_dataset = dsets.MNIST(root ='./data',  
                            train = True,  
                            transform = transforms.ToTensor(), 
                            download = True) 
  
test_dataset = dsets.MNIST(root ='./data',  
                           train = False,  
                           transform = transforms.ToTensor()) 

# Hyper Parameters  
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,  
                                           batch_size = batch_size,  
                                           shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False) 

In [4]:
# 우리가 사용할 class 지정
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 
  
    def forward(self, x): 
        out = self.linear(x) 
        return out 

In [5]:
model = LogisticRegression(input_size, num_classes) # 모델 정의

criterion = nn.CrossEntropyLoss() # loss 함수 설정
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) # optimizer 설정

In [8]:
# Training the Model 
for epoch in range(num_epochs): 
    for i, (images, labels) in enumerate(train_loader): 
        images = Variable(images.view(-1, 28 * 28)) 
        labels = Variable(labels) 
  
        # Forward + Backward + Optimize 
        optimizer.zero_grad()  # Gradient: starts from zero
        outputs = model(images)  ### forward pass
        loss = criterion(outputs, labels)  ### calculating loss 
        loss.backward()  ### backward propagation
        optimizer.step()  ### updating weight
  
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch+1, num_epochs, i+1, len(train_dataset) 
                     // batch_size, loss.data))

Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.2124
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 2.0889
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 2.0536
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.9415
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.8689
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.8263
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.7235
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.6325
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.6378
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.6044
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.5570
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.4624
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.4384
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.4294
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.3291
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.2936
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.3345
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.3444
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.2063
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1916
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [10]:
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
  
print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct // total)) 

Accuracy of the model on the 10000 test images:  83 %


---

## Q4-1) optim.SGD를 사용하지 않고 코드 구현

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [11]:
# 데이터 
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]])

In [14]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
W=torch.zeros(1)
b=torch.zeros(1)

In [16]:
nb_epochs = 1000 # 전체 데이터셋 대상 SGD 반복 횟수 지정
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train * W + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2) # Cost function 정의

    ### SGD func. 쓰지 않고 gradient 계산식 지정 
    w_grad = torch.sum(((W * x_train + b) - y_train) * x_train)
    b_grad = torch.sum((W * x_train + b) - y_train)
    
    # manually update W and b (optimizer 함수를 쓰지 않고 cost func. 편미분하여 update)
    W -= lr * w_grad
    b -= lr * b_grad

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()))

Epoch    0/1000 W: 0.140, b: 0.060 Cost: 4.666667
Epoch  100/1000 W: 0.887, b: 0.256 Cost: 0.009462
Epoch  200/1000 W: 0.922, b: 0.178 Cost: 0.004594
Epoch  300/1000 W: 0.945, b: 0.124 Cost: 0.002231
Epoch  400/1000 W: 0.962, b: 0.087 Cost: 0.001083
Epoch  500/1000 W: 0.973, b: 0.060 Cost: 0.000526
Epoch  600/1000 W: 0.982, b: 0.042 Cost: 0.000255
Epoch  700/1000 W: 0.987, b: 0.029 Cost: 0.000124
Epoch  800/1000 W: 0.991, b: 0.020 Cost: 0.000060
Epoch  900/1000 W: 0.994, b: 0.014 Cost: 0.000029
Epoch 1000/1000 W: 0.996, b: 0.010 Cost: 0.000014


A) 간단히 말하면, 기존 GD 방식은 one-by-one 으로 계산하여 시간이 오래 걸리므로,  
SGD는 모델이 wide and deep 할 때 데이터를 mini-batch로 쪼개서 이런 단점을 보완한다.  
하지만 우리가 가진 data는 굳이 batch를 더 쪼갤 필요성이 없으므로 직접 지정한 수식으로 update해도 충분하다. 

---

## Q4-2) 다른 optimizer 적용해보기

### Adam

In [22]:
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor(([1],[2],[3]))

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

optimizer = optim.Adam(model.parameters(), lr=0.001)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: 0.278, b: 0.050 Cost: 2.296805
Epoch  100/1000 W: 0.375, b: 0.146 Cost: 1.487315
Epoch  200/1000 W: 0.460, b: 0.231 Cost: 0.919919
Epoch  300/1000 W: 0.534, b: 0.303 Cost: 0.544140
Epoch  400/1000 W: 0.596, b: 0.361 Cost: 0.310156
Epoch  500/1000 W: 0.646, b: 0.407 Cost: 0.174275
Epoch  600/1000 W: 0.686, b: 0.441 Cost: 0.101120
Epoch  700/1000 W: 0.717, b: 0.463 Cost: 0.064650
Epoch  800/1000 W: 0.739, b: 0.476 Cost: 0.047645
Epoch  900/1000 W: 0.756, b: 0.481 Cost: 0.039950
Epoch 1000/1000 W: 0.768, b: 0.480 Cost: 0.036249


### Adadelta

In [23]:
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor(([1],[2],[3]))

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

optimizer = optim.Adadelta(model.parameters(), lr=0.001)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: 0.365, b: -0.390 Cost: 3.021716
Epoch  100/1000 W: 0.366, b: -0.389 Cost: 3.017332
Epoch  200/1000 W: 0.366, b: -0.389 Cost: 3.011763
Epoch  300/1000 W: 0.367, b: -0.388 Cost: 3.005242
Epoch  400/1000 W: 0.367, b: -0.387 Cost: 2.997899
Epoch  500/1000 W: 0.368, b: -0.387 Cost: 2.989824
Epoch  600/1000 W: 0.369, b: -0.386 Cost: 2.981086
Epoch  700/1000 W: 0.370, b: -0.385 Cost: 2.971736
Epoch  800/1000 W: 0.371, b: -0.384 Cost: 2.961820
Epoch  900/1000 W: 0.372, b: -0.383 Cost: 2.951376
Epoch 1000/1000 W: 0.373, b: -0.382 Cost: 2.940437


### RMSprop

In [25]:
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor(([1],[2],[3]))

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

optimizer = optim.RMSprop(model.parameters(), lr=0.001)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: 0.155, b: 0.006 Cost: 3.425576
Epoch  100/1000 W: 0.336, b: 0.186 Cost: 1.608132
Epoch  200/1000 W: 0.432, b: 0.281 Cost: 0.951961
Epoch  300/1000 W: 0.513, b: 0.360 Cost: 0.537302
Epoch  400/1000 W: 0.585, b: 0.429 Cost: 0.276849
Epoch  500/1000 W: 0.648, b: 0.485 Cost: 0.131319
Epoch  600/1000 W: 0.699, b: 0.523 Cost: 0.066789
Epoch  700/1000 W: 0.736, b: 0.535 Cost: 0.046477
Epoch  800/1000 W: 0.763, b: 0.515 Cost: 0.039202
Epoch  900/1000 W: 0.788, b: 0.468 Cost: 0.031981
Epoch 1000/1000 W: 0.818, b: 0.404 Cost: 0.023767
